In [ ]:
# %pip install tweepy
# %pip install Sastrawi
# %pip install numpy
# %pip install sklearn
# %pip install joblib
# %pip install pickle
%pip install textblob

In [ ]:
import tweepy 

BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAJQrYQEAAAAA1BtlexYIrHRZvqkz6TEOraEC%2B0k%3D5cOp6U7ytEMpcTDctDQWVl6HFd0Y28pxhDnTGyHA9KiiC1MVCF'

client = tweepy.Client(bearer_token=BEARERTOKEN)

query = '"Ruang Guru" lang:id'

# get data max 100
responses = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at'])
tweet100 = [tweet for tweet in responses.data]

# get max 1000
tweettexts = [{'id' : tweet.id, 'text' :tweet.text} for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=1000)]


In [7]:
import re
import sys

def Cleansing(tweet):
    # remove old style retweet text "RT"
    new_tweet = re.sub(r'RT\s', '', tweet)
    
    # hilangkan username
    new_tweet = re.sub(r'^@*[A-Za-z0-9]+', '', new_tweet)
    
    
    # remove hyperlinks
    new_tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', new_tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    new_tweet = re.sub(r'^@*[A-Za-z0-9]+', '', new_tweet)

    new_tweet = re.sub("[^A-Za-z" "]+", " ", new_tweet)
    
    return new_tweet.strip().lower()

cleantweets = []
for tweet in tweettexts:
    tweet = Cleansing(tweet['text'])
    cleantweets.append(tweet)

In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stem
stemmedtexts   = [stemmer.stem(text) for text in cleantweets]

## Bersih Labeling

In [10]:
# kata_positif = open('kata_positif.txt', 'r').readlines()
kata_negatif = open('kata_negatif.txt', 'r').readlines()

with open('kata_negatif_2.txt', 'w') as kat:
    for kata in kata_negatif:
        kat.write(f'{stemmer.stem(kata)}\n')
    kat.close()


